In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import gzip
from sklearn.preprocessing import OneHotEncoder
from scipy.special import expit
import celluloid
from celluloid import Camera
from matplotlib import animation
from IPython.display import HTML
from matplotlib.lines import Line2D

np.random.seed(2021)

def gradient_descent(xinit,steps,gradient):
    """Run gradient descent.
    Return an array with the rows as the iterates.
    """
    xs = [xinit]
    x = xinit
    for step in steps:
        x = x - step*gradient(x)
        xs.append(x)
    return np.array(xs)

def nagd(winit,gradient,eta=0.1,nsteps=100):
    """Run Nesterov's accelrated graident descent.
    Return an array with the rows as the iterates.
    """
    ws = [winit]
    u = v = w = winit    
    for i in range(nsteps):
        etai = (i+1)*eta/2
        alphai = 2/(i+3)
        w = v - eta*gradient(v)
        u = u - etai*gradient(v)
        v = alphai*u + (1-alphai)*w
        ws.append(w)
    return np.array(ws)

def animated_lplot(Ys,title,labels=['1','2','3','4','5','6'],ylabel='Function value', xlabel='Iteration', name='animation.gif'):
    """Animated line plot of the Y values.
    Ys is a list where each element is an array of numbers to plot.
    """
    colors = ['blue','red','green','black','cyan','purple','pink']
    fig, ax = plt.subplots(figsize=(6,6))
    camera = Camera(fig)
    T = len(Ys[0])
    plt.yscale('log')
    for t in range(T):
        for j in range(len(Ys)):
            plt.plot(range(t),Ys[j][:t],color=colors[j],marker='o')
        camera.snap()
    handles = []
    for i in range(len(Ys)):
        handles.append(Line2D([0], [0], color=colors[i], label=labels[i]))
    plt.legend(handles = handles, loc = 'upper right')
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    animation = camera.animate(interval=100,blit=False)
    plt.close()
    animation.save(name, writer='PillowWriter', fps=10)
    return animation

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def generate_logistic(n,d):
    """Generate a dataset under the logistic model (see Assignment 1).
    """
    X = np.random.randn(n,d)
    wstar = np.random.randn(d,1)
    wstar = wstar/np.linalg.norm(wstar)

    preds = sigmoid(np.dot(X,wstar))
    y = np.ones((n,1))
    y[preds > np.random.rand(n,1)] = 0
    return (X,y,wstar)

def generate_lsr(n,d):
    X = np.random.normal(0, 1, (n, d))
    wstar = np.random.normal(0, 1, d)
    noise = np.random.normal(0, 0.1, n)
    y = X.dot(wstar) + noise
    return(X,y,wstar)

In [2]:
# TODO: write the correct functions
def lr_cost(X,y,w):
    n,d = X.shape
    return (1/n)*sum(-y*np.log(sigmoid(X.dot(w)))-((1-y)*np.log(1-sigmoid(X.dot(w)))))

def lr_gradient(X,y,w):
    n,d = X.shape
    return (1/n) * X.T.dot(sigmoid(X.dot(w))-y)


# # for lsr
def lsr_cost(A, b, x):
    """Least squares objective."""
    return (0.5/n) * np.linalg.norm(A.dot(x)-b)**2

def lsr_gradient(A, b, x):
    """Gradient of least squares objective at x."""
    return A.T.dot(A.dot(x)-b)/n

In [3]:
# generating dataset
n,d = 1000, 20
dataset = generate_logistic(n,d)

In [4]:
# use step sizes 0.01, 0.05, 0.1 for 350 iterations
X = dataset[0]
y = dataset[1].flatten()

objective = lambda w: lr_cost(X, y, w)
gradient = lambda w: lr_gradient(X, y, w)

w0 = np.random.normal(0, 1, d)

In [5]:
gd_1 = gradient_descent(w0, [0.025]*350, gradient)
nagd_1 = nagd(w0,gradient,0.01,350)

gd_2 = gradient_descent(w0, [0.05]*350, gradient)
nagd_2 = nagd(w0,gradient,0.05,350)

gd_3 = gradient_descent(w0, [0.075]*350, gradient)
nagd_3 = nagd(w0,gradient,0.1,350)

In [6]:
animated_lplot(Ys=[[objective(w) for w in gd_1],[objective(w) for w in nagd_1]],labels=['GD','NAGD'], \
               title='Function value vs Iteration, Step Size=0.025', name='gd_1_fVal.gif')
animated_lplot(Ys=[[np.linalg.norm(dataset[2]-w) for w in gd_1],[np.linalg.norm(dataset[2]-w) for w in nagd_1]],\
               labels=['GD','NAGD'], ylabel="Distance", title='w* Distance vs Iteration, Step Size=0.025', \
               name='gd_1_w.gif')

animated_lplot(Ys=[[objective(w) for w in gd_2],[objective(w) for w in nagd_2]],labels=['GD','NAGD'], \
               title='Function value vs Iteration, Step Size=0.05', name='gd_2_fVal.gif')
animated_lplot(Ys=[[np.linalg.norm(dataset[2]-w) for w in gd_2],[np.linalg.norm(dataset[2]-w) for w in nagd_2]],\
               labels=['GD','NAGD'], ylabel="Distance", title='w* Distance vs Iteration, Step Size=0.05', \
               name='gd_2_w.gif')

animated_lplot(Ys=[[objective(w) for w in gd_3],[objective(w) for w in nagd_3]],labels=['GD','NAGD'], \
               title='Function value vs Iteration, Step Size=0.075', name='gd_3_fVal.gif')
animated_lplot(Ys=[[np.linalg.norm(dataset[2]-w) for w in gd_3],[np.linalg.norm(dataset[2]-w) for w in nagd_3]],\
               labels=['GD','NAGD'], ylabel="Distance", title='w* Distance vs Iteration, Step Size=0.075', \
               name='gd_3_w.gif')



MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.


In [7]:
animated_lplot(Ys=[[objective(w) for w in gd_1],[objective(w) for w in gd_2],[objective(w) for w in gd_3],\
                   [objective(w) for w in nagd_1],[objective(w) for w in nagd_2],[objective(w) for w in nagd_3]],\
               labels=['GD; Step=0.025','GD; Step=0.05','GD; Step=0.075','NAGD; Step=0.025','NAGD; Step=0.05',\
                       'NAGD; Step=0.075'], title='Function value vs Iteration', name='All_fVal.gif')

animated_lplot(Ys=[[np.linalg.norm(dataset[2]-w) for w in gd_1],[np.linalg.norm(dataset[2]-w) for w in gd_2],\
                   [np.linalg.norm(dataset[2]-w) for w in gd_3],[np.linalg.norm(dataset[2]-w) for w in nagd_1],\
                   [np.linalg.norm(dataset[2]-w) for w in nagd_2],[np.linalg.norm(dataset[2]-w) for w in nagd_3]],\
               labels=['GD; Step=0.025','GD; Step=0.05','GD; Step=0.075','NAGD; Step=0.025','NAGD; Step=0.05',\
                       'NAGD; Step=0.075'], ylabel="Distance", title='w* Distance vs Iteration', name='All_w.gif')


MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.


In [8]:
animated_lplot(Ys=[[objective(w) for w in gd_1],[objective(w) for w in gd_2],[objective(w) for w in gd_3]],\
               labels=['GD; Step=0.025','GD; Step=0.05','GD; Step=0.075'], title='Function value vs Iteration, GD', \
               name='gd_All_fVal.gif')

animated_lplot(Ys=[[objective(w) for w in nagd_1],[objective(w) for w in nagd_2],[objective(w) for w in nagd_3]],\
               labels=['NAGD; Step=0.025','NAGD; Step=0.05','NAGD; Step=0.075'], \
               title='Function value vs Iteration, NAGD', name='nagd_All_fVal.gif')

animated_lplot(Ys=[[np.linalg.norm(dataset[2]-w) for w in gd_1],[np.linalg.norm(dataset[2]-w) for w in gd_2],\
                   [np.linalg.norm(dataset[2]-w) for w in gd_3]],labels=['GD; Step=0.025','GD; Step=0.05',\
                                                                         'GD; Step=0.075'], ylabel="Distance", \
               title='w* Distance vs Iteration, GD', name='gd_All_w.gif')

animated_lplot(Ys=[[np.linalg.norm(dataset[2]-w) for w in nagd_1],[np.linalg.norm(dataset[2]-w) for w in nagd_2],\
                   [np.linalg.norm(dataset[2]-w) for w in nagd_3]],labels=['NAGD; Step=0.025','NAGD; Step=0.05',\
                                                                           'NAGD; Step=0.075'], ylabel="Distance", \
               title='w* Distance vs Iteration, NAGD', name='nagd_All_w.gif')


MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
MovieWriter PillowWriter unavailable; using Pillow instead.
